In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.font_manager as fm
import matplotlib.patches as patches

from matplotlib.dates import DateFormatter

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import FuncFormatter

from sklearn.metrics import silhouette_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler

from unittest.mock import inplace
from IPython.display import set_matplotlib_formats
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split, KFold
from random import randint

from scipy.io import loadmat
import seaborn as sns
import glob
import torch.nn as nn
import torch
import numpy as np
import pandas as pd
import datetime
import math
import time
import datetime
import warnings


warnings.simplefilter(action='ignore', category=FutureWarning)
# plt.rcParams["font.weight"] = "bold"
# plt.rcParams["axes.labelweight"] = "bold"
warnings.filterwarnings(action='ignore')
#그래프에서 음수 값이 나올 때, 깨지는 현상 방지
mpl.rc('axes',unicode_minus=False)

# Korean Font Setting
plt.rcParams["font.family"] = 'Malgun Gothic'

In [2]:
def extract_discharge(battery):
    mat = loadmat(battery + '.mat')
    dataset = []
    capacity_data = []
    
    for i in range(len(mat[battery][0, 0]['cycle'][0])):
        row = mat[battery][0, 0]['cycle'][0, i]
        if row['type'][0] == 'discharge':
            data = row['data']
            capacity = data[0][0]['Capacity'][0][0]
            capacity_data.append(capacity)

    cap = np.array(capacity_data)
    cap = cap[cap != 0]
    if cap.shape[0]>=310:
        cap = np.delete(cap, 310)
    else:
        pass
    return cap

In [3]:
def extract_discharge_B0018(battery):
    mat = loadmat(battery + '.mat')
    dataset = []
    capacity_data = []
    
    for i in range(len(mat[battery][0, 0]['cycle'][0])):
        row = mat[battery][0, 0]['cycle'][0, i]
        if row['type'][0] == 'discharge':
            data = row['data']
            capacity = data[0][0]['Capacity'][0][0]
            capacity_data.append(capacity)

    cap = np.array(capacity_data)
    cap = cap[cap != 0]
    return cap



In [5]:
def extract_charge_preprocessing_cnn_plus_lstm(battery):
  mat = loadmat(battery + '.mat')
  print('Total data in dataset: ', len(mat[battery][0, 0]['cycle'][0]))
  charInput = []

  for i in range(len(mat[battery][0, 0]['cycle'][0])-1):
    try:
      row = mat[battery][0, 0]['cycle'][0, i]
      
      if row['type'][0] == 'charge':
        le = len(row['data']['Voltage_measured'][0][0][0]) % 30
        vTemp = row['data']['Voltage_measured'][0][0][0][:] if le ==0 else row['data']['Voltage_measured'][0][0][0][:-le] 
        if vTemp.shape[0] ==0:
          continue
        vTemp = vTemp.reshape(-1, len(vTemp)//30)
        vTemp = np.mean(vTemp,axis=1)

        iTemp = row['data']['Current_measured'][0][0][0][:] if le ==0 else row['data']['Current_measured'][0][0][0][:-le]   
        iTemp = iTemp.reshape(-1, len(iTemp)//30)
        iTemp = np.mean(iTemp,axis=1)

        tTemp = row['data']['Temperature_measured'][0][0][0][:] if le ==0 else row['data']['Temperature_measured'][0][0][0][:-le]
        tTemp = tTemp.reshape(-1, len(tTemp)//30)
        tTemp = np.mean(tTemp,axis=1)            
        charInput.append(np.concatenate([vTemp, iTemp, tTemp]))
        
    except Exception as e:
      print(i)

      print(e)

  charInput = np.array(charInput)
  charInput = np.delete(charInput, [22, 83], axis=0)
  charInput = charInput.reshape(167, -1, 30)

  charInput = charInput[~np.any(charInput == 0, axis=(1, 2))]
  return charInput


In [57]:
def extract_charge_preprocessing_cnn_plus_lstm_B0018(battery):
  mat = loadmat(battery + '.mat')
  print('Total data in dataset: ', len(mat[battery][0, 0]['cycle'][0]))
  charInput = []
  charInput = np.zeros((len(mat[battery][0, 0]['cycle'][0])-1,3,30))
  for i in range(len(mat[battery][0, 0]['cycle'][0])-1):
    try:
      row = mat[battery][0, 0]['cycle'][0, i]
      
      if row['type'][0] == 'charge':
        le = len(row['data']['Voltage_measured'][0][0][0]) % 30
        vTemp = row['data']['Voltage_measured'][0][0][0][:] if le ==0 else row['data']['Voltage_measured'][0][0][0][:-le] 
        if vTemp.shape[0] ==0:
          continue
        vTemp = vTemp.reshape(-1, len(vTemp)//30)
        vTemp = np.mean(vTemp,axis=1)
        vTemp = vTemp.reshape(1, -1)

        iTemp = row['data']['Current_measured'][0][0][0][:] if le ==0 else row['data']['Current_measured'][0][0][0][:-le]   
        iTemp = iTemp.reshape(-1, len(iTemp)//30)
        iTemp = np.mean(iTemp,axis=1)
        iTemp = iTemp.reshape(1, -1)

        tTemp = row['data']['Temperature_measured'][0][0][0][:] if le ==0 else row['data']['Temperature_measured'][0][0][0][:-le]
        tTemp = tTemp.reshape(-1, len(tTemp)//30)
        tTemp = np.mean(tTemp,axis=1)    
        tTemp = tTemp.reshape(1, -1)     

        charInput[i,:, :] = np.concatenate([vTemp, iTemp, tTemp])
        # charInput.append(np.concatenate([vTemp, iTemp, tTemp]))
        # print(np.array(charInput).shape)
        
    except Exception as e:
      print(i)
      print(e)

  charInput = np.array(charInput)
  charInput = np.delete(charInput, [114, 137], axis=0)
  charInput = charInput[~np.any(charInput == 0, axis=(1, 2))]

  return charInput

In [58]:

charInput18 = extract_charge_preprocessing_cnn_plus_lstm_B0018('B0018')

Total data in dataset:  319


In [48]:
charInput5 = extract_charge_preprocessing_cnn_plus_lstm('B0005')
charInput6 = extract_charge_preprocessing_cnn_plus_lstm('B0006')
charInput7 = extract_charge_preprocessing_cnn_plus_lstm('B0007')


Total data in dataset:  616
Total data in dataset:  616
Total data in dataset:  616


In [60]:
cap5 = extract_discharge('B0005')
cap6 = extract_discharge('B0006')
cap7 = extract_discharge('B0007')
cap18 = extract_discharge_B0018('B0018')


In [65]:
InitC5 = cap5[0]
InitC6 = cap6[0]
InitC7 = cap7[0]
InitC18 = cap18[0]

In [77]:
cap = np.hstack([cap6.reshape(1,-1), cap5.reshape(1,-1), cap7.reshape(1,-1), cap18.reshape(1,-1)])
charInput = np.concatenate((charInput6, charInput5, charInput7, charInput18), axis=0)

In [103]:
def minmax_norm_cnn_plus_lstm(charInput, cap):
    r = np.max(charInput, axis=2, keepdims=True) - np.min(charInput, axis=2, keepdims=True)
    xData = (charInput - np.min(charInput, axis=2, keepdims=True)) / r
    yData = cap
    ym = np.min(yData)
    yr = np.max(yData) - ym
    yData = (yData - ym) / yr

    return xData, yData.reshape(-1,1), ym, yr

In [104]:
xB, yB, ym, yr = minmax_norm_cnn_plus_lstm(charInput, cap)